In [2]:
from nltk.corpus import gutenberg
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [3]:
text=''
for txt in gutenberg.fileids():
    if 'shakespeare' in txt:
        text+=gutenberg.raw(txt).lower()
chars=sorted(list(set(text)))

In [4]:
char_indices=dict((c,i) for i,c in enumerate(chars))
indices_char=dict((i,c) for i,c in enumerate(chars))

In [5]:
"语料库长度:{}，基本字符集长度：{}".format(len(text),len(chars))

'语料库长度:375542，基本字符集长度：50'

In [6]:
maxlen=40
step=1
sentences=[]
next_chars=[]
for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print('nb seq:{}'.format(len(sentences)))

nb seq:375502


In [7]:
print(sentences[0:5])
print(next_chars[0:5])


['[the tragedie of julius caesar by willia', 'the tragedie of julius caesar by william', 'he tragedie of julius caesar by william ', 'e tragedie of julius caesar by william s', ' tragedie of julius caesar by william sh']
['m', ' ', 's', 'h', 'a']


In [8]:
import numpy as np
def onehot_encode_ger(sentence,char_indices,maxlen=40):
    X=np.zeros((maxlen,len(char_indices.keys())))
    for t,char in enumerate(sentence):
        X[t,char_indices[char]]=1
    return X
    
def onehot_encode_label(char,char_indices):
    Y=np.zeros((len(char_indices)))
    Y[char_indices[char]]=1
    return Y

In [9]:
def data_generator(datastore,next_char,char_indices,batchsize=32,maxlen=40):
    X,Y=[],[]
    while True:
        for i in range(len(datastore)):
            if (i % batchsize==0 and X and Y) or (i==len(datastore)):
                X=np.reshape(X,(len(X),maxlen,len(char_indices)))
                Y=np.reshape(Y,(len(Y),len(char_indices)))
                yield X,Y
                X,Y=[],[]
            x=datastore[i]
            y=next_chars[i]
            x=onehot_encode_ger(x,char_indices,maxlen=maxlen)
            y=onehot_encode_label(y,char_indices)   
            X.append(x)
            Y.append(y)

In [10]:
print(next_chars[0])
print(char_indices)
onehot_encode_label(next_chars[0],char_indices)

m
{'\n': 0, ' ': 1, '!': 2, '&': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '9': 17, ':': 18, ';': 19, '?': 20, '[': 21, ']': 22, 'a': 23, 'b': 24, 'c': 25, 'd': 26, 'e': 27, 'f': 28, 'g': 29, 'h': 30, 'i': 31, 'j': 32, 'k': 33, 'l': 34, 'm': 35, 'n': 36, 'o': 37, 'p': 38, 'q': 39, 'r': 40, 's': 41, 't': 42, 'u': 43, 'v': 44, 'w': 45, 'x': 46, 'y': 47, 'z': 48, 'æ': 49}


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [11]:
sd=onehot_encode_ger(sentences[0],char_indices)

In [12]:
sd.shape

(40, 50)

In [13]:
print(char_indices)
print(np.sum(sd,axis=0))
print(sentences[0])


{'\n': 0, ' ': 1, '!': 2, '&': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '9': 17, ':': 18, ';': 19, '?': 20, '[': 21, ']': 22, 'a': 23, 'b': 24, 'c': 25, 'd': 26, 'e': 27, 'f': 28, 'g': 29, 'h': 30, 'i': 31, 'j': 32, 'k': 33, 'l': 34, 'm': 35, 'n': 36, 'o': 37, 'p': 38, 'q': 39, 'r': 40, 's': 41, 't': 42, 'u': 43, 'v': 44, 'w': 45, 'x': 46, 'y': 47, 'z': 48, 'æ': 49}
[0. 6. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 4.
 1. 1. 1. 4. 1. 1. 1. 4. 1. 0. 3. 0. 0. 1. 0. 0. 2. 2. 2. 2. 0. 1. 0. 1.
 0. 0.]
[the tragedie of julius caesar by willia


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,LSTM,Dropout,GRU
from tensorflow.keras.optimizers import RMSprop
model=Sequential()
model.add(GRU(128,input_shape=(maxlen,len(chars))))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer=RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=optimizer)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 128)               69120     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                6450      
_________________________________________________________________
activation (Activation)      (None, 50)                0         
Total params: 75,570
Trainable params: 75,570
Non-trainable params: 0
_________________________________________________________________


In [15]:
import math
batch_size=512
epochs=6
model_structure=model.to_json()
with open('smodel/shakes_lstm_model.json','w') as json_file:
    json_file.write(model_structure)
for i in range(90):
    print('第{}次'.format(i+1))
    history=model.fit(data_generator(sentences,next_chars,char_indices,batchsize=32,maxlen=40),steps_per_epoch=math.ceil(len(sentences)/batch_size),epochs=1)
    model.save_weights('smodel/shakes_lstm_weight_{}.h5'.format(i+1))


第1次
734/734 [==============================] - 8s 2ms/step - loss: 2.4404 - accuracy: 0.3100
第2次
734/734 [==============================] - 2s 2ms/step - loss: 1.9507 - accuracy: 0.4329
第3次
734/734 [==============================] - 2s 2ms/step - loss: 1.8564 - accuracy: 0.4587
第4次
734/734 [==============================] - 2s 2ms/step - loss: 1.8090 - accuracy: 0.4728
第5次
734/734 [==============================] - 2s 2ms/step - loss: 1.7868 - accuracy: 0.4805
第6次
734/734 [==============================] - 2s 2ms/step - loss: 1.7885 - accuracy: 0.4792
第7次
734/734 [==============================] - 2s 2ms/step - loss: 1.7952 - accuracy: 0.4774
第8次
734/734 [==============================] - 2s 2ms/step - loss: 1.8065 - accuracy: 0.4736
第9次
734/734 [==============================] - 2s 2ms/step - loss: 1.8267 - accuracy: 0.4670
第10次
734/734 [==============================] - 2s 2ms/step - loss: 1.8523 - accuracy: 0.4615
第11次
734/734 [==============================] - 2s 2ms/step - loss: 1

In [16]:
sds=data_generator(sentences,next_chars,char_indices,batchsize=32,maxlen=40)

In [17]:
next(sds)

(array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],


In [17]:
import random
def sampple(preds,temperature=1.0):
    preds=np.array(preds).astype('float64')
    preds=np.log(preds)/temperature
    exp_preds=np.exp(preds)
    preds=exp_preds/np.sum(exp_preds)
    probas=np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [51]:
from tensorflow.keras.models import model_from_json
with open('smodel/shakes_lstm_model.json','r') as json_file:
    j_s=json_file.read()
modelX=model_from_json(j_s)
modelX.summary()
modelX.load_weights('smodel/shakes_lstm_weight_8.h5')



Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_7 (GRU)                  (None, 128)               69120     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                6450      
_________________________________________________________________
activation_4 (Activation)    (None, 50)                0         
Total params: 75,570
Trainable params: 75,570
Non-trainable params: 0
_________________________________________________________________


In [35]:
preds.shape

(1, 50)

In [52]:
import sys
start_index=random.randint(0,len(text)-maxlen-1)
for diversity in [0.2,0.3,0.4,0.5,0.1,0.6,1.0]:
    print()
    print('引入新奇度diversity:{}'.format(diversity))
    generated=''
    sentences=text[start_index:start_index+maxlen]
    generated+=sentences
    print('生成文本的起点：{}'.format(generated))
    sys.stdout.write(generated)
    for i in range(400):
        x=np.zeros((1,maxlen,len(chars)))
        for t,char in enumerate(sentences):
            x[0,t,char_indices[char]]=1
        preds=modelX.predict(x,verbose=0)[0]
        next_index=sampple(preds,temperature=diversity)
        next_char=indices_char[next_index]
        generated+=next_char
        sentences=sentences[1:]+next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


引入新奇度diversity:0.2
生成文本的起点：and promise of their mettle:

low march 
and promise of their mettle:

low march man, and whe hast shoure and these these strome to send
that sere this will will a man, and when,
that man to selfe.
that hast should not the sare to man, and were these these to selfe
that a man to man to these world be and why say farth streses if is a man,
that will seans inder these this the war,
now these that i world and when the sere,
these that will shake to see the sare these these these 

引入新奇度diversity:0.3
生成文本的起点：and promise of their mettle:

low march 
and promise of their mettle:

low march so stire and what seake a then thise this geresting my stront and farth,
and when these these share that will feare,
that with should strenter the would caska, and should this this the man

   cask. i would wor man, and whe his a these these these to sean,
that as i will these that will man i warke,
and whe sare a wart so strong a this these to man,
what say fares, and when the 

In [68]:
start_index=random.randint(0,len(text)-maxlen-1)
sentences=text[start_index:start_index+maxlen]
x=np.zeros((1,maxlen,len(chars)))
for t,char in enumerate(sentences):
    x[0,t,char_indices[char]]=1
preds=modelX.predict(x,verbose=0)

In [69]:
preds=preds[0]
preds

array([1.3380340e-02, 2.4428939e-02, 3.0075779e-15, 4.4806177e-12,
       7.4759722e-03, 1.8878028e-08, 2.4904981e-08, 5.8379300e-02,
       3.5610752e-05, 2.9412573e-02, 2.2766442e-18, 1.6471226e-17,
       2.2499488e-18, 2.1927373e-18, 2.2913865e-18, 6.5899006e-18,
       2.3156388e-18, 7.7540266e-18, 5.3861374e-03, 6.5950776e-04,
       3.6670347e-03, 2.2857991e-18, 9.1086358e-18, 1.3988922e-02,
       5.6121605e-03, 6.7085579e-02, 1.0153474e-02, 1.7607534e-01,
       1.5715145e-03, 5.7660630e-03, 1.1308195e-02, 5.5813767e-02,
       2.3330172e-18, 2.0412235e-02, 1.0090580e-01, 1.9799618e-02,
       5.0031878e-03, 2.4540836e-02, 1.2248082e-02, 1.0890204e-05,
       1.1204369e-02, 1.4168358e-01, 1.2137576e-01, 3.2445908e-02,
       2.3286061e-04, 1.1044219e-02, 3.7534028e-06, 8.8883732e-03,
       1.7463359e-11, 2.1341638e-16], dtype=float32)

In [70]:
preds=np.array(preds).astype('float64')
    # preds=np.array(preds).astype('float64')
    # preds=np.log(preds)/temperature
    # exp_preds=np.exp(preds)
    # preds=exp_preds/np.sum(exp_preds)
    # probas=np.random.multinomial(1,preds,1)
    # return np.argmax(probas)

In [71]:
preds

array([1.33803403e-02, 2.44289394e-02, 3.00757789e-15, 4.48061770e-12,
       7.47597218e-03, 1.88780280e-08, 2.49049812e-08, 5.83792999e-02,
       3.56107521e-05, 2.94125732e-02, 2.27664424e-18, 1.64712257e-17,
       2.24994884e-18, 2.19273731e-18, 2.29138646e-18, 6.58990061e-18,
       2.31563877e-18, 7.75402663e-18, 5.38613740e-03, 6.59507758e-04,
       3.66703467e-03, 2.28579906e-18, 9.10863580e-18, 1.39889224e-02,
       5.61216054e-03, 6.70855790e-02, 1.01534743e-02, 1.76075339e-01,
       1.57151453e-03, 5.76606300e-03, 1.13081951e-02, 5.58137670e-02,
       2.33301722e-18, 2.04122346e-02, 1.00905798e-01, 1.97996181e-02,
       5.00318781e-03, 2.45408360e-02, 1.22480821e-02, 1.08902041e-05,
       1.12043694e-02, 1.41683578e-01, 1.21375762e-01, 3.24459076e-02,
       2.32860606e-04, 1.10442191e-02, 3.75340278e-06, 8.88837315e-03,
       1.74633589e-11, 2.13416383e-16])

In [72]:
preds=np.log(preds)/0.2
preds
    # preds=np.array(preds).astype('float64')
    # preds=np.log(preds)/temperature
    # exp_preds=np.exp(preds)
    # preds=exp_preds/np.sum(exp_preds)
    # probas=np.random.multinomial(1,preds,1)
    # return np.argmax(probas)

array([ -21.56984396,  -18.55993403, -167.18820665, -130.656301  ,
        -24.48030556,  -88.92633565,  -87.54099003,  -14.20396953,
        -51.2143147 ,  -17.63166518, -203.11914571, -193.22458357,
       -203.17812097, -203.30690499, -203.086873  , -197.80496703,
       -203.03423049, -196.991597  ,  -26.11963387,  -36.62008412,
        -28.04185967, -203.09908007, -196.18654361,  -21.34744762,
        -25.91409755,  -13.50893088,  -22.94969669,   -8.68421656,
        -32.27857729,  -25.77882877,  -22.41113794,  -14.4286736 ,
       -202.99684651,  -19.45810412,  -11.46783943,  -19.61046316,
        -26.48840004,  -18.53708387,  -22.01192959,  -57.13823441,
        -22.45725727,   -9.77079514,  -10.54432037,  -17.14090479,
        -41.8253527 ,  -22.52924072,  -62.46423861,  -23.61505622,
       -123.85458104, -180.41643283])

In [73]:
exp_preds=np.exp(preds)
exp_preds

array([4.28880000e-10, 8.70007637e-09, 2.46084587e-73, 1.80588234e-57,
       2.33527689e-11, 2.39763535e-39, 9.58144675e-39, 6.78101046e-07,
       5.72671711e-23, 2.20122839e-08, 6.11611794e-89, 1.21235439e-84,
       5.76584840e-89, 5.06912536e-89, 6.31672120e-89, 1.24278020e-86,
       6.65815748e-89, 2.80308612e-86, 4.53301485e-12, 1.24766945e-16,
       6.63093976e-13, 6.24008129e-89, 6.26998769e-86, 5.35699570e-10,
       5.56737444e-12, 1.35876973e-06, 1.07912901e-10, 1.69235968e-04,
       9.58499742e-15, 6.37377774e-12, 1.84912580e-10, 5.41634945e-07,
       6.91177700e-89, 3.54366578e-09, 1.04611786e-05, 3.04287465e-09,
       3.13497460e-12, 8.90116323e-09, 2.75638858e-10, 1.53172249e-25,
       1.76578201e-10, 5.70949355e-05, 2.63426733e-05, 3.59583400e-08,
       6.84668137e-19, 1.64314192e-10, 7.44947825e-28, 5.54767990e-11,
       1.62419772e-54, 4.42729823e-79])

In [74]:
preds=exp_preds/np.sum(exp_preds)
preds

array([1.61355985e-06, 3.27319854e-05, 9.25835220e-70, 6.79420637e-54,
       8.78592852e-08, 9.02053751e-36, 3.60479336e-35, 2.55119525e-03,
       2.15454224e-19, 8.28160265e-05, 2.30104512e-85, 4.56119744e-81,
       2.16926447e-85, 1.90713886e-85, 2.37651737e-85, 4.67566737e-83,
       2.50497471e-85, 1.05459504e-82, 1.70543992e-08, 4.69406204e-13,
       2.49473469e-09, 2.34768341e-85, 2.35893499e-82, 2.01544328e-06,
       2.09459332e-08, 5.11205064e-03, 4.05996837e-07, 6.36710415e-01,
       3.60612921e-11, 2.39798355e-08, 6.95689969e-07, 2.03777373e-03,
       2.60039308e-85, 1.33322067e-05, 3.93577170e-02, 1.14480982e-05,
       1.17946025e-08, 3.34885273e-05, 1.03702619e-06, 5.76274461e-22,
       6.64333834e-07, 2.14806229e-01, 9.91080953e-02, 1.35284773e-04,
       2.57590238e-15, 6.18193394e-07, 2.80269049e-24, 2.08718372e-07,
       6.11066084e-51, 1.66566654e-75])

In [93]:
probas=np.random.multinomial(1,preds,1)
np.argmax(probas)

27

In [76]:
probas

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0]])

In [77]:
np.argmax(preds)

27